In [1]:
import simpleaudio as sa
from pydub import AudioSegment
import pandas as pd
import os
import time
import requests
import numpy as np
import gradio as gr
import pydub
import simpleaudio
import librosa
import speech_recognition as spr #to translate speech to text

In [2]:
from transformers import pipeline
asr = pipeline("automatic-speech-recognition", "facebook/wav2vec2-base-960h")
classifier = pipeline("text-classification")
from googletrans import Translator
translator = Translator()

2022-08-02 19:29:48.902387: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-08-02 19:29:48.902690: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Metal device set to: Apple M1


Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english)


In [4]:
def button_update():
    return gr.update(variant='primary')

def text_to_audio(audio):
    text = asr(audio)["text"]
    translation = translator.translate(text, dest='mi')
    tranlated_text=translation.text
    if 'GOD' in text:
        concept="Yes!"     
    else:
        concept="No!"
        
    sentiment=classifier(text)[0]["label"]
    
    if sentiment=="POSITIVE" and concept=="No!":
        filename = ('data/intro.wav')
        
    elif sentiment=="POSITIVE" and concept=="Yes!":
        filename = ('data/learnabtgods.wav')
    
    else:
        filename = ('data/KiaOra.wav')
        
    y, sr = librosa.load(filename)
    return gr.update(value='To test again clear the input below (x) and Submit again..'),gr.update(visible=True),text,tranlated_text,sentiment,concept,(sr, y)
    

with gr.Blocks(css="""

body {
    background: rgb(2,0,36);
    background: linear-gradient(180deg, rgba(2,0,36,1) 0%, rgba(7,51,99,1) 70%, rgba(6,3,17,1) 100%); 
}

""") as demo:
    with gr.Row(visible=True):
        gr.Image('Te Ipukarea logo.png',interactive=False,image_mode='L',label=' ').style(height=54, width=150)
        gr.Image('AUT Maori.jpg',interactive=False,label=' ').style(height=54, width=150)
        #gr.Image('Tui.jpg',interactive=False,label=' ').style(height=54, width=150)
    gr.Markdown("""
                
                """)
    #gr.Markdown("**Record from microphone** and then **Submit** to see the output.")
    with gr.Row():
        with gr.Column():
            lbl1=gr.Label('Click Record from microphone below and then click Submit button to see the output..')
            inp1 = gr.Audio(source="microphone",type="filepath",label='Record and Submit',visible=True)
            btn1 = gr.Button("Submit", variant='secondary')
            gr.Image('Tui.jpg',interactive=False,label=' ').style(height=54, width=450)
        with gr.Column(visible=False) as outs:
            out1 = gr.Textbox(label="Transcripted:")
            out2 = gr.Textbox(label="Translated:")
            out3 = gr.Label(label="Sentiment Analysed:")
            out4 = gr.Textbox(label="Any domain/concept identified, eg. God?")
            out5 = gr.Audio()
              
    inp1.change(fn=button_update, inputs=None, outputs=[btn1])
    btn1.click(fn=text_to_audio, inputs=inp1, outputs=[lbl1,outs,out1,out2,out3,out4,out5])
    
demo.launch(share=True)



Running on local URL:  http://127.0.0.1:7861/
Running on public URL: https://35929.gradio.app

This share link expires in 72 hours. For free permanent hosting, check out Spaces: https://huggingface.co/spaces


(<gradio.routes.App at 0x2b50f4f40>,
 'http://127.0.0.1:7861/',
 'https://35929.gradio.app')

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


/Users/manjuvallayil/miniforge3/envs/TeReo-gpu/lib/python3.9/site-packages/gradio/processing_utils.py:161: UserWarning: Audio data is not in 16-bit integer format.Trying to convert to 16-bit int format.
  warnings.warn(
